In [1]:
import polars as pl
import polars_ds as pds
import numpy as np

# This notebook illustrates the basic usage of this package

You need to create an environment with this package installed to run this notebook. (v0.3.4+)

## New in v0.3.4

You can now access the following queries by calling them directly from pds (e.g. pds.query_lstsq, etc.)

1. All knn related queries
2. All lstsq (linear regression) related queries
3. All graph related queries
4. Most common metrics
5. Miscallenous, which are self-explanatory by the names

More will be added if appropriate. For most occasions, in pds.func() arguments, you can use either str or pl.Expr. For example:

```python
df.select(
    pds.query_lstsq_report(
        pl.col("x1"), "x2", # str | pl.Expr
        target = "y",
        add_bias=False
    ).alias("report")
).unnest("report")

shape: (2, 5)
┌──────────┬───────┬────────────┬────────────┬───────┐
│ feat_idx ┆ coeff ┆ std_err    ┆ t          ┆ p>|t| │
│ ---      ┆ ---   ┆ ---        ┆ ---        ┆ ---   │
│ u16      ┆ f64   ┆ f64        ┆ f64        ┆ f64   │
╞══════════╪═══════╪════════════╪════════════╪═══════╡
│ 0        ┆ 2.0   ┆ 2.3854e-16 ┆ 8.3842e15  ┆ 0.0   │
│ 1        ┆ -1.0  ┆ 9.0158e-17 ┆ -1.1092e16 ┆ 0.0   │
└──────────┴───────┴────────────┴────────────┴───────┘

```

# New in v0.3.5

It is much easier to generate randoms now.

```python
import polars_ds as pds

df = pds.random_data(size=100_000, n_cols = 1).select(
    pds.random(0.0, 12.0).alias("uniform_1"),
    pds.random_exp(0.5).alias("exp"),
    pds.random_normal(0.0, 1.0).alias("normal"),
)
```

# Num Extensions

In [2]:
size = 10_000
df = pl.DataFrame({
    "f": np.sin(list(range(size)))
    , "time_idx": range(size)
    , "dummy": ["a"] * (size // 2) + ["b"] * (size // 2)
    , "a": np.random.random(size = size)
    , "b": np.random.random(size = size)
    , "x1" : range(size)
    , "x2" : range(size, size + size)
    , "y": range(-size, 0)
    , "actual": np.round(np.random.random(size=size)).astype(np.int32)
    , "predicted": np.random.random(size=size)
    , "dummy_groups":["a"] * (size//2) + ["b"] * (size//2) 
})
df.head()

f,time_idx,dummy,a,b,x1,x2,y,actual,predicted,dummy_groups
f64,i64,str,f64,f64,i64,i64,i64,i32,f64,str
0.0,0,"""a""",0.040527,0.970016,0,10000,-10000,1,0.285598,"""a"""
0.841471,1,"""a""",0.081158,0.521181,1,10001,-9999,1,0.857457,"""a"""
0.909297,2,"""a""",0.976482,0.515597,2,10002,-9998,1,0.181328,"""a"""
0.14112,3,"""a""",0.079779,0.695853,3,10003,-9997,1,0.116835,"""a"""
-0.756802,4,"""a""",0.95303,0.466074,4,10004,-9996,0,0.900835,"""a"""


In [3]:
# Column-wise Jaccard Similarity. Result should be 0 as they are distinct
df.select(
    pds.query_jaccard_col("x1", pl.col("x2"))
)

x1
f64
0.0


In [4]:
# FFT. First is real part, second is complex part
# By default, this behaves the same as np's rfft, which returns a non-redundant 
# compact representation of fft output.
df.select(
    pds.rfft("f")
).head()

f
"array[f64, 2]"
"[1.939505, 0.0]"
"[1.939506, 0.000209]"
"[1.939508, 0.000418]"
"[1.939512, 0.000627]"
"[1.939518, 0.000835]"


In [5]:
# FFT. But return the full length
df.select(
    pds.rfft("f", return_full=True)
).shape

(10000, 1)

In [6]:
# Convolution (by FFT). 
# Modes: `same`, `left` (left-aligned same), `right` (right-aligned same), `valid` or `full`
# Currently slower than SciPy but provides parallelism because of Polars
df.select(
    pds.convolve("f", [-1, 0, 0, 0, 1], mode = "full"),
    pds.convolve("a", [-1, 0, 0, 0, 1], mode = "full"),
    pds.convolve("b", [-1, 0, 0, 0, 1], mode = "full"),
).head()

f,a,b
f64,f64,f64
1.3944e-15,-0.040527,-0.970016
-0.841471,-0.081158,-0.521181
-0.909297,-0.976482,-0.515597
-0.14112,-0.079779,-0.695853
0.756802,-0.912503,0.503942


In [7]:
# Least Square (Linear Regression)
df.select(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

y
list[f64]
"[2.0, -1.0]"


In [8]:
df.select(
    pds.query_lstsq_report(
        # str | pl.Expr
        "x1", "x2",
        target = pl.col("y"),
        add_bias=False
    ).alias("report")
).unnest("report")

feat_idx,coeff,std_err,t,p>|t|
u16,f64,f64,f64,f64
0,2.0,2.3854e-16,8.3842e15,0.0
1,-1.0,9.0158e-17,-1.1092e16,0.0


In [9]:
df.lazy().select(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = "y", # We can either put pl.col("y") here or just the string "y"
        add_bias=False
    )
).collect()

y
list[f64]
"[2.0, -1.0]"


In [10]:
df.select(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    ).over(pl.col("dummy"))
).head() 

coeffs
list[f64]
"[2.0, -1.0]"
"[2.0, -1.0]"
"[2.0, -1.0]"
"[2.0, -1.0]"
"[2.0, -1.0]"


In [11]:
# If you want prediction and residue instead of coefficients
df.select(
    "x1",
    "x2",
    "y",
    pds.query_lstsq(
        "x1", pl.col("x2"),
        target = "y",
        add_bias=False, 
        return_pred=True
    ).alias("prediction")
).unnest("prediction").head()

x1,x2,y,pred,resid
i64,i64,i64,f64,f64
0,10000,-10000,-10000.0,5.8208e-11
1,10001,-9999,-9999.0,5.8208e-11
2,10002,-9998,-9998.0,5.8208e-11
3,10003,-9997,-9997.0,5.8208e-11
4,10004,-9996,-9996.0,5.8208e-11


In [12]:
df.group_by("dummy").agg(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

dummy,coeffs
str,list[f64]
"""a""","[2.0, -1.0]"
"""b""","[2.0, -1.0]"


In [13]:
# Rolling regression, kind of slow rn
df.lazy().rolling(
    index_column = pl.col("time_idx").set_sorted(),
    period = "30i",
    # offset = "-1i"
).agg(
    pds.query_lstsq(pl.col("x1"), pl.col("x2"), target = pl.col("y"), add_bias=False).alias("coefficients")
).slice(offset = 30).select(
    "time_idx",
    "coefficients",
).collect().head(10)

time_idx,coefficients
i64,list[f64]
30,"[2.0, -1.0]"
31,"[2.0, -1.0]"
32,"[2.0, -1.0]"
33,"[2.0, -1.0]"
34,"[2.0, -1.0]"
35,"[2.0, -1.0]"
36,"[2.0, -1.0]"
37,"[2.0, -1.0]"
38,"[2.0, -1.0]"


In [14]:
# Conditional Entropy, should be 0 because x1 is an ID
df.select(
    pds.query_cond_entropy("y", "x1")
)

y
f64
-0.0


# ML Metrics

In [15]:
df.group_by("dummy_groups").agg(
    pl.col("actual").metric.l2_loss(pl.col("predicted")).alias("l2"),
    pl.col("actual").metric.log_loss(pl.col("predicted")).alias("log loss"),
    # Or equivalently pl.col("actual").metric.binary_metrics_combo(pl.col("predicted"))
    pds.query_binary_metrics(actual="actual", pred="predicted").alias("combo")
).unnest("combo")


dummy_groups,l2,log loss,precision,recall,f,average_precision,roc_auc
str,f64,f64,f64,f64,f64,f64,f64
"""b""",0.336203,1.005318,0.5,0.493249,0.248301,0.501396,0.495637
"""a""",0.331565,0.997816,0.519093,0.513577,0.25816,0.5097,0.501246


# Str Extension

In [16]:
size = 100_000
df2 = pl.DataFrame({
    "sen":["Hello, world! I'm going to church."] * size,
    "word":["words", "word"] * (size //2)
})
df2.head()

sen,word
str,str
"""Hello, world! …","""words"""
"""Hello, world! …","""word"""
"""Hello, world! …","""words"""
"""Hello, world! …","""word"""
"""Hello, world! …","""words"""


In [17]:
# Tokenize
df2.select(
    pl.col("sen").str.to_lowercase().str2.tokenize().explode().unique()
)

sen
str
"""church"""
"""to"""
"""hello"""
"""going"""
"""world"""


In [18]:
df2.select(
    pl.col("sen").str.to_lowercase().str2.tokenize(stem=True).explode().unique()
)

sen
str
"""go"""
"""church"""
"""world"""
""""""
"""hello"""


In [19]:
df2.select(
    pl.col("word").str2.levenshtein("world")
).head()

word
u32
2
1
2
1
2


In [20]:
# Damerau-Levenshtein
df2.select(
    pl.col("word").str2.d_levenshtein("world")
).head()

word
u32
2
1
2
1
2


In [21]:
df2.select(
    pl.col("word").str2.levenshtein("world", return_sim = True)
).head()

word
f64
0.6
0.8
0.6
0.8
0.6


In [22]:
df2.filter(
    # This is way faster than computing ditance and then doing a filter
    pds.filter_by_levenshtein(pl.col("word"), "world", 1) # <= 1. 
).head()

sen,word
str,str
"""Hello, world! …","""word"""
"""Hello, world! …","""word"""
"""Hello, world! …","""word"""
"""Hello, world! …","""word"""
"""Hello, world! …","""word"""


In [23]:
df = pl.DataFrame({
    "word":["apple", "banana", "pineapple", "asasasas", "sasasass"],
    "other_data": [1,2,3,4,5]
})
gibberish = ["asasasa", "sasaaasss", "asdasadadfa"]

In [24]:
df.filter(
    pl.col("word").str2.similar_to_vocab(
        vocab = gibberish,
        threshold = 0.5,
        metric = "lv", # Levenshtein similarity. Other options: dleven, osa, jw
        strategy = "any" # True if the word is similar to any word in vocab. Other options: "all", "avg"
    )
)

word,other_data
str,i64
"""asasasas""",4
"""sasasass""",5


In [25]:
df.select(
    pl.col("word").str2.levenshtein("asasasa", return_sim=True).alias("asasasa"),
    pl.col("word").str2.levenshtein("sasaaasss", return_sim=True).alias("sasaaasss"),
    pl.col("word").str2.levenshtein("asdasadadfa", return_sim=True).alias("asdasadadfa"),
    pl.col("word").str2.fuzz("apples").alias("LCS based Fuzz match - apples"),
    pl.col("word").str2.osa("apples", return_sim = True).alias("Optimal String Alignment - apples"),
    pl.col("word").str2.jw("apples").alias("Jaro-Winkler - apples"),
)


asasasa,sasaaasss,asdasadadfa,LCS based Fuzz match - apples,Optimal String Alignment - apples,Jaro-Winkler - apples
f64,f64,f64,f64,f64,f64
0.142857,0.111111,0.090909,0.833333,0.833333,0.966667
0.428571,0.333333,0.272727,0.166667,0.0,0.444444
0.111111,0.111111,0.090909,0.555556,0.444444,0.5
0.875,0.666667,0.545455,0.25,0.25,0.527778
0.75,0.777778,0.454545,0.25,0.25,0.527778


# Stats Extension

In [26]:
import numpy as np

df = pl.DataFrame({
    "a": [None, None] + list(np.random.normal(size = 998))
})
df.head()

a
f64
null
null
1.092547
-0.313788
0.910145


In [27]:
# Genenrate random numbers, respecting null positions in reference column (pl.col("a"))
df.with_columns(
    pl.col("a").stats.rand_normal(mean = 0.5, std = 1., respect_null=True).alias("random")
).head()

a,random
f64,f64
null,null
null,null
1.092547,0.118862
-0.313788,0.915701
0.910145,0.506431


In [28]:
# Genenrate random string
df.with_columns(
    pl.col("a").stats.rand_str(min_size = 1, max_size = 5, respect_null=True).alias("random_str")
).head()

a,random_str
f64,str
null,null
null,null
1.092547,"""ocV"""
-0.313788,"""NZK"""
0.910145,"""VDUt"""


In [29]:
# Genenrate fixed size random string, while respecting column a's nulls
df.with_columns(
    pl.col("a").stats.rand_str(min_size = 5, max_size = 5, respect_null=True).alias("random_str")
).head()

a,random_str
f64,str
null,null
null,null
1.092547,"""ZPOG1"""
-0.313788,"""s0DFP"""
0.910145,"""E3wpq"""


In [30]:
df.with_columns(
    # Sample from a normal distribution, using reference column "a" 's mean and std
    pl.col("a").stats.rand_normal().alias("test1") 
    # Sample from uniform distribution, with low = 0 and high = "a"'s max, and respect the nulls in "a"
    , pl.col("a").stats.rand_uniform(low = 0., high = None, respect_null=True).alias("test2")
).with_columns(
    # Add a random pertubation to test1
    pds.perturb("test1", epsilon=0.001).alias("test1_perturbed")
).head()

a,test1,test2,test1_perturbed
f64,f64,f64,f64
null,0.608435,null,0.608179
null,-0.98947,null,-0.989521
1.092547,0.603246,0.639586,0.602959
-0.313788,-1.644565,0.791304,-1.644321
0.910145,1.888639,2.199168,1.888938


In [31]:
# New in v0.3.5
# This way, we don't have a reference column, so we cannot respect nulls, but is more convenient to use.
df.with_columns(
    pds.random().alias("[0, 1)"),
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("Normal"),
    pds.random_int(0, 10).alias("Int from [0, 10)"),
).head()

a,"[0, 1)",Normal,"Int from [0, 10)"
f64,f64,f64,i32
null,0.16185,-0.022253,6
null,0.282183,2.524618,7
1.092547,0.335685,0.256993,4
-0.313788,0.69809,0.935573,1
0.910145,0.979704,-1.020565,5


In [32]:
# Genenrate 2 random sample, both normally distributed
# Run Welch's t test on them, p value should be big since they have equal mean
# Run a normality test. Again, p value should be big since they are normally distributed 

df.with_columns(
    pds.random_normal(0.5, 1.0).alias("test1"),
    pds.random_normal(0.5, 2.0).alias("test2"),
).select(
    pds.query_ttest_ind("test1", "test2", equal_var=False).alias("t-test"),
    pds.normal_test("test1").alias("normality_test")
).select(
    pl.col("t-test").struct.field("statistic").alias("t-tests: statistics")
    , pl.col("t-test").struct.field("pvalue").alias("t-tests: pvalue")
    , pl.col("normality_test").struct.field("statistic").alias("normality_test: statistics")
    , pl.col("normality_test").struct.field("pvalue").alias("normality_test: pvalue")
)

t-tests: statistics,t-tests: pvalue,normality_test: statistics,normality_test: pvalue
f64,f64,f64,f64
2.461335,0.013957,0.463269,0.793236


In [33]:
size = 5_000
df = pl.DataFrame({
    "market_id": range(size),
}).with_columns(
    pl.col("market_id").mod(3),
    var1 = pds.random(),
    var2 = pds.random(),
    category_1 = pds.random_int(0, 5),
    category_2 = pds.random_int(0, 10),
)

df.head(5)

market_id,var1,var2,category_1,category_2
i64,f64,f64,i32,i32
0,0.594209,0.661919,3,5
1,0.757217,0.158478,3,7
2,0.32018,0.274224,1,6
0,0.490232,0.444415,0,9
1,0.349611,0.465001,4,7


In [34]:
# In dataframe statistical tests!
df.select(
    pds.query_ttest_ind("var1", "var2", equal_var=True).alias("t-test"),
    pds.query_chi2("category_1", "category_2").alias("chi2-test"),
    pds.query_f_test("var1", group = "category_1").alias("f-test")
)

t-test,chi2-test,f-test
struct[2],struct[2],struct[2]
"{0.638694,0.523037}","{42.537894,0.21018}","{0.654591,0.623602}"


In [35]:
# Can also be done in group by context
df.group_by("market_id").agg(
    pds.query_ttest_ind("var1", "var2", equal_var=False).alias("t-test"),
    pds.query_chi2("category_1", "category_2").alias("chi2-test"),
    pds.query_f_test("var1", group = "category_1").alias("f-test")
)

market_id,t-test,chi2-test,f-test
i64,struct[2],struct[2],struct[2]
0,"{0.333412,0.738844}","{43.815521,0.173839}","{1.626642,0.164931}"
1,"{0.083359,0.933571}","{29.776028,0.758276}","{0.117724,0.976252}"
2,"{0.694302,0.487541}","{29.672917,0.762562}","{0.574279,0.681332}"


# Nearest Neighbors Related Tasks

These queries can be very slow when data/dimension gets huge, even when processed in parallel.

In [36]:
import polars_ds as pds
size = 2000
df = pl.DataFrame({
    "id": range(size), 
}).with_columns(
    pds.random().alias("var1"),
    pds.random().alias("var2"),
    pds.random().alias("var3"),
    pds.random().alias("r"),
    (pds.random() * 10).alias("rh"),
    pl.col("id").cast(pl.UInt32)
)

In [37]:
# Get neighbor count. The point itself is always considered a neighbor to itself.
df.with_columns(
    pds.query_nb_cnt(
        0.1, # radius 
        pl.col("var1"), "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        dist = "inf", # L Infinity distance 
        parallel = True 
    ).alias("nb_l_inf_cnt")
).head() 

id,var1,var2,var3,r,rh,nb_l_inf_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.855798,0.237808,0.441565,0.403929,6.479341,20
1,0.227444,0.176608,0.165496,0.573626,0.271922,22
2,0.963625,0.379795,0.459282,0.931296,6.595991,21
3,0.164434,0.160854,0.898112,0.02794,6.844658,13
4,0.144249,0.934727,0.095495,0.518644,5.906987,19


In [38]:
df.with_columns(
    pds.query_nb_cnt(
        pl.col("r"), # radius be an expression too
        "var1", "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        dist = "l1", # L 1 distance 
        parallel = True 
    ).alias("nb_l1_r_cnt")
).head()

id,var1,var2,var3,r,rh,nb_l1_r_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.855798,0.237808,0.441565,0.403929,6.479341,154
1,0.227444,0.176608,0.165496,0.573626,0.271922,296
2,0.963625,0.379795,0.459282,0.931296,6.595991,889
3,0.164434,0.160854,0.898112,0.02794,6.844658,1
4,0.144249,0.934727,0.095495,0.518644,5.906987,147


In [39]:
# Get ids of the k nearest neighbors. 
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = "id",  # pl.col("id"), str | pl.Expr
        k = 3, 
        dist = "l2", # squared l2
        parallel = True
    ).alias("best friends")
).head() 

id,var1,var2,var3,r,rh,best friends
u32,f64,f64,f64,f64,f64,list[u32]
0,0.855798,0.237808,0.441565,0.403929,6.479341,"[0, 83, … 407]"
1,0.227444,0.176608,0.165496,0.573626,0.271922,"[1, 1624, … 194]"
2,0.963625,0.379795,0.459282,0.931296,6.595991,"[2, 929, … 1914]"
3,0.164434,0.160854,0.898112,0.02794,6.844658,"[3, 1977, … 1102]"
4,0.144249,0.934727,0.095495,0.518644,5.906987,"[4, 796, … 431]"


In [40]:
# Get all neighbors within radius r
# The point itself is always considered a neighbor to itself.
print(df.select(
    pl.col("id"),
    pds.query_radius_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = pl.col("id"),
        r = 0.1, 
        dist = "l2", # actually this is squared l2
        parallel = True
    ).alias("best friends"),
).with_columns( # -1 to remove the point itself
    (pl.col("best friends").list.len() - 1).alias("best friends count")
).head())

shape: (5, 3)
┌─────┬───────────────────┬────────────────────┐
│ id  ┆ best friends      ┆ best friends count │
│ --- ┆ ---               ┆ ---                │
│ u32 ┆ list[u32]         ┆ u32                │
╞═════╪═══════════════════╪════════════════════╡
│ 0   ┆ [0, 83, … 670]    ┆ 204                │
│ 1   ┆ [1, 1624, … 1340] ┆ 184                │
│ 2   ┆ [2, 929, … 1500]  ┆ 149                │
│ 3   ┆ [3, 1977, … 192]  ┆ 137                │
│ 4   ┆ [4, 796, … 398]   ┆ 106                │
└─────┴───────────────────┴────────────────────┘


In [41]:
# Get ids of the k nearest neighbors and distances
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = pl.col("id"),
        k = 3, 
        dist = "l2", # actually this is squared l2
        parallel = True,
        return_dist = True
    ).alias("best_friends_w_dist")
).unnest("best_friends_w_dist").head()

id,var1,var2,var3,r,rh,idx,dist
u32,f64,f64,f64,f64,f64,list[u32],list[f64]
0,0.855798,0.237808,0.441565,0.403929,6.479341,"[0, 83, … 407]","[0.0, 0.002975, … 0.00588]"
1,0.227444,0.176608,0.165496,0.573626,0.271922,"[1, 1624, … 194]","[0.0, 0.001998, … 0.004466]"
2,0.963625,0.379795,0.459282,0.931296,6.595991,"[2, 929, … 1914]","[0.0, 0.002418, … 0.004338]"
3,0.164434,0.160854,0.898112,0.02794,6.844658,"[3, 1977, … 1102]","[0.0, 0.002745, … 0.007421]"
4,0.144249,0.934727,0.095495,0.518644,5.906987,"[4, 796, … 431]","[0.0, 0.000797, … 0.003012]"


In [42]:
# Filter to only points near the given point
df.filter(
    pds.query_within_dist_from(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5, 0.5],
        r = 0.2,
        dist = "l2" # actually this is squared l2, so this is asking for squared l2 <= 0.2
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
0,0.855798,0.237808,0.441565,0.403929,6.479341
8,0.727479,0.347359,0.608749,0.610317,7.536041
12,0.453981,0.379954,0.895441,0.164987,7.295983
13,0.376001,0.544418,0.266029,0.136613,8.518468
16,0.54988,0.15123,0.611696,0.020818,8.540414


In [43]:
# Haversine distance is available when dimension is 2
df.filter(
    pds.query_within_dist_from(
        pl.col("var1"), pl.col("var2"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5],
        r = 10, # in km
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
28,0.463971,0.436025,0.319795,0.775136,4.672099
36,0.54394,0.436549,0.802354,0.14605,6.091761
59,0.476374,0.578844,0.296306,0.487146,0.13915
101,0.432539,0.47039,0.082891,0.132899,8.059089
138,0.525352,0.459083,0.015951,0.617515,7.193868


In [44]:
df.filter(
    pds.query_within_dist_from(
        pl.col("var1"), pl.col("var2"), 
        pt = [0.5, 0.5],
        # radius can also be an existing column in the dataframe.
        r = pl.col("rh"), 
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
138,0.525352,0.459083,0.015951,0.617515,7.193868
225,0.488199,0.474461,0.76333,0.121684,5.944395
260,0.522097,0.492712,0.19169,0.473217,4.931974
351,0.482834,0.567561,0.868282,0.883075,8.127492
419,0.450922,0.427618,0.622492,0.432098,9.75776


In [45]:
friends = df.select(
    pl.col("id").cast(pl.UInt64),
    pds.query_radius_ptwise(
        # Columns used as the coordinates in n-d space
        pl.col("var1"), pl.col("var2"), 
        index=pl.col("id"),
        r = 0.02, 
        dist = "l2",
    ).alias("friends")
).with_columns(
    pl.col("friends").list.len().alias("count")
)
friends.head()

id,friends,count
u64,list[u32],u32
0,"[0, 1213, … 863]",115
1,"[1, 459, … 1729]",129
2,"[2, 1823, … 1242]",88
3,"[3, 1888, … 924]",128
4,"[4, 753, … 1475]",102


# Simple Graph Queries

There is limited functionality in the Graph module currently. E.g. Only constant cost per edge.

Graph queries are very expensive.

In [46]:
# friends.select(
#     pl.col("friends").graph.eigen_centrality() # .arg_max()
# ).head()

In [47]:
# Turn friends to a table suitable for graph analytics
df_graph = friends.select(
    pl.col("id"),
    pl.col("friends"),
).explode(pl.col("friends")).with_columns(
    pl.col("id").cast(pl.UInt32),
    pl.col("friends").cast(pl.UInt32),
)
df_graph.head()

id,friends
u32,u32
0,0
0,1213
0,1412
0,1440
0,545


In [48]:
df_graph.select(
    # Shortest path to the node with id = 3
    # Node and link can be str | pl.Expr
    pds.query_shortest_path(node = "id", link = pl.col("friends"), target = 3, cost = None, parallel=True).alias("shortest_path")
).unnest("shortest_path").sort("id")

id,path
u32,list[u32]
0,"[863, 105, … 3]"
1,[3]
2,"[599, 1700, … 3]"
3,[]
4,"[1475, 1665, … 3]"
…,…
1995,"[1265, 1017, 3]"
1996,"[31, 1331, … 3]"
1997,"[1756, 725, … 3]"


In [49]:
df_graph.select(
    # Almost every node can reach node 3, and the number is the number steps to reach it
    # This is a way faster way to filter results if you don't need the actual path
    pl.col("id").graph.reachable(link="friends", target = 3).alias("reach")
).unnest("reach")

id,reachable,steps
u32,bool,u32
796,true,6
1667,true,5
747,true,2
19,true,5
1171,true,7
…,…,…
607,true,6
447,true,3
542,true,3


In [50]:
relationships = pl.DataFrame({
    "id": range(5),
    "connections":[[1,2,3,4], [2,3], [4], [0,1,2], [1]],
    # Small values means closer
    "close-ness":[[0.4, 0.3, 0.2, 0.1], [0.1, 1.0], [0.5], [0.1, 0.1, 0.1], [0.1]]
}).with_columns(
    pl.col("id").cast(pl.UInt32),
    pl.col("connections").list.eval(pl.element().cast(pl.UInt32))
).explode(
    pl.col("connections"), pl.col("close-ness")
)

relationships.head(50)

id,connections,close-ness
u32,u32,f64
0,1,0.4
0,2,0.3
0,3,0.2
0,4,0.1
1,2,0.1
…,…,…
2,4,0.5
3,0,0.1
3,1,0.1


In [51]:
# To go to node at id = 1, node 0 would rather go to 4 first and then 1.
relationships.select(
    pl.col("id").graph.shortest_path(
        link = "connections",
        target = 1,
        cost = pl.col("close-ness"),
        parallel = False
    ).alias("path")
).unnest("path").head()

id,path,cost
u32,list[u32],f64
4,[1],0.1
3,[1],0.1
0,"[4, 1]",0.2
1,[],0.0
2,"[4, 1]",0.6


In [52]:
# In and out deg
relationships.select(
    pl.col("id").graph.in_out_deg(link=pl.col("connections")).alias("deg")
).unnest("deg")

node,in,out
u32,u32,u32
4,2,1
3,2,3
0,1,4
1,3,2
2,3,1


# String Nearest Neighbors

This might be very slow for very large vocab / column.

In [53]:
df = pl.DataFrame({
    "a":["AAAAA", "ABCABC", "AAAADDD", "ADSDSDS", "WORD"],
    "b":["AAAAT", "ABCACD", "ADSSD", "APPLES", "WORLD"] 
})

In [54]:
# Use Levenshtein to find the nearest neighbor in vocab to word in column a
df.select(
    pl.col("a").str2.similar_words(
        vocab = pl.col("b"),
        k = 1,
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
str
"""AAAAT"""
"""ABCACD"""
"""AAAAT"""
"""ADSSD"""
"""WORLD"""


In [55]:
# Use Levenshtein to find 2 nearest neighbors
df.select(
    pl.col("a").str2.similar_words(
        vocab = pl.col("b"),
        k = 2,
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
list[str]
"[""AAAAT"", ""ADSSD""]"
"[""ABCACD"", ""AAAAT""]"
"[""AAAAT"", ""ABCACD""]"
"[""ADSSD"", ""APPLES""]"
"[""WORLD"", ""ADSSD""]"


In [56]:
# Currently only Levenshtein and hamming are implemented for this
# Empty means nothing in vocab can be compared in the hamming sense with the corresponding word in a
df.select(
    pl.col("a").str2.similar_words(
        vocab = pl.col("b"),
        k = 2,
        threshold = 4, # <= threshold hamming distance away
        metric = "hamming"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
list[str]
"[""AAAAT"", ""ADSSD""]"
"[""ABCACD""]"
[]
[]
[]


In [57]:
# You may provide a vocab like this
df.select(
    pl.col("a"),
    pl.col("a").str2.similar_words(
        vocab = ["WORLD", "AAAAA", "ABCDEFG", "ZIV", "TQQQ"],
        k = 3,
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

a,similar_words_from_vocab
str,list[str]
"""AAAAA""","[""AAAAA"", ""ZIV"", ""WORLD""]"
"""ABCABC""","[""ABCDEFG"", ""AAAAA"", ""ZIV""]"
"""AAAADDD""","[""AAAAA"", ""WORLD"", ""ABCDEFG""]"
"""ADSDSDS""","[""ABCDEFG"", ""WORLD"", ""AAAAA""]"
"""WORD""","[""WORLD"", ""ZIV"", ""TQQQ""]"
